In [1]:
#r "nuget:Octokit"
#r "nuget:Microsoft.Data.Analysis"
#r "nuget:XPlot.Plotly"

In [1]:
using Microsoft.Data.Analysis;
using Octokit;
using XPlot.Plotly;
using System.Linq;

In [1]:
using Microsoft.AspNetCore.Html;
Formatter.Register<DataFrame>((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }

    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));

    writer.Write(t);
}, "text/html");

In [1]:
var github = new GitHubClient(new ProductHeaderValue("GH-Stats"));

In [1]:
// var startTime = new DateTime(2019, 07, 01, 00, 00, 00);
// var timeOffset = new DateTimeOffset(startTime, TimeZoneInfo.Local.GetUtcOffset(startTime));
// var issueRequest = new RepositoryIssueRequest
// {
//     Since = timeOffset,
//     SortDirection = SortDirection.Ascending
// };
// var wcfIssues = await github.Issue.GetAllForRepository("dotnet", "wcf", issueRequest);
var wcfIssues = await github.Issue.GetAllForRepository("dotnet", "wcf");

In [1]:
var createdAtColumn = new PrimitiveDataFrameColumn<DateTimeOffset>("CreatedAt");
var closedAtColumn = new PrimitiveDataFrameColumn<DateTimeOffset>("ClosedAt");
var titleColumn = new StringDataFrameColumn("Title", 0);
foreach (var issue in wcfIssues)
{
    createdAtColumn.Append(issue.CreatedAt);
    closedAtColumn.Append(issue.ClosedAt);
    titleColumn.Append(issue.Title);
}
var df = new DataFrame(titleColumn, createdAtColumn, closedAtColumn);

In [1]:
var startTime = new DateTime(2020, 07, 01, 00, 00, 00);
var starttimeOffset = new DateTimeOffset(startTime, TimeZoneInfo.Local.GetUtcOffset(startTime));
// binSize = 1 week
var binSize = (int) (DateTimeOffset.UtcNow - starttimeOffset).TotalDays/7;
display(Chart.Plot(
    new Graph.Histogram()
    {
        x = createdAtColumn.Filter(timeOffset, DateTimeOffset.MaxValue),
        nbinsx = binSize
    }
))